In [4]:
import pandas as pd
import warnings

In [5]:
warnings.filterwarnings("ignore")

### Carregando o dataset

In [ ]:
df = pd.read_csv('../data/raw_data/goodreads.csv', sep=';')

### Verificando valores duplicados

Como o objetivo futuro será criar um modelo para prever a média das avaliações de usuários para um determinado livro, precisamos verificar se existe alguma linha com valores de `book_id` duplicados, visto que dois livros iguais com dados diferentes seria um problema para etapas futuras.

In [7]:
df.duplicated(subset=['book_id']).sum()

np.int64(0)

Não há nenhuma instância duplicada, por isso não é necessário nenhum tratamento.

### Dividindo o dataset em Treino, Validação e Teste

Antes de realizar o tratamento de valores ausentes em um dataset, é fundamental dividir os conjuntos de treino, validação e teste. Isso é importante porque tratar valores ausentes antes da divisão pode introduzir vazamento de dados, o que ocorre quando informações do conjunto de teste influenciam os dados de treino, resultando em uma avaliação enviesada do modelo que será treinado no futuro.

In [8]:
def split_data(df : pd.DataFrame, train_frac: float = 0.5, val_frac: float = 0.25, 
               test_frac: float = 0.25) -> list[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Split a DataFrame into training, validation, and test sets.

    Parameters:
    df: The DataFrame to be split.
    train_frac: The fraction of the data to include in the training set.
    val_frac: The fraction of the data to include in the validation set.
    test_frac: The fraction of the data to include in the test set.

    Returns:
    A list containing three DataFrames: the training set, the validation set, and the test set.
    """

    assert train_frac + val_frac + test_frac == 1

    df = df.sample(frac=1, replace=False).reset_index(drop=True)

    qtd_lines = df.shape[0]

    train = df.iloc[:int(qtd_lines * train_frac)]
    validation = df.iloc[int(qtd_lines * train_frac) : int(qtd_lines * (1-test_frac))]
    test = df.iloc[int(qtd_lines * (1-test_frac)):]

    return train, validation, test

In [9]:
df_train, df_val, df_test = split_data(df, train_frac=0.6, val_frac=0.2, test_frac=0.2)


### Verificando valores ausentes

In [10]:
# Verificando quais colunas possuem valores nulos
nan_columns = df_train.isnull().sum()
nan_columns = nan_columns[nan_columns > 0]

print(nan_columns)

language_code       21685
book_format         18152
num_pages           16109
publication_year    15857
book_genre            561
author_name             1
dtype: int64


Analisando as colunas que possuem valores nulos é perceptível que as colunas `language_code`, `book_format`, `num_pages`, `publication_year` e `book_genre` possuem uma grande quantidade de valores nulos, por isso devem ser tratadas de forma diferente das demais colunas.

Para tratar as colunas com valores numéricos (`num_pages` e `publication_year`) é possível utilizar a média de todos os valores **do conjunto de treino** (para evitar vazamento de dados) dessa coluna, no entanto para as colunas com valores categóricos é necessário fazer o tratamento de forma diferente.

In [11]:
# Substituindo os valores nulos das colunas numéricas pela média dos demais valores
df_train['num_pages'] = df_train['num_pages'].fillna(df_train['num_pages'].mean()).astype('int64')
df_val['num_pages'] = df_val['num_pages'].fillna(df_train['num_pages'].mean()).astype('int64')
df_test['num_pages'] = df_test['num_pages'].fillna(df_train['num_pages'].mean()).astype('int64')

df_train['publication_year'] = df_train['publication_year'].fillna(df_train['publication_year'].mean()).astype('int64')
df_val['publication_year'] = df_val['publication_year'].fillna(df_train['publication_year'].mean()).astype('int64')
df_test['publication_year'] = df_test['publication_year'].fillna(df_train['publication_year'].mean()).astype('int64')

In [12]:
# Verificando a quantidade de valores distintos nas colunas categóricas
print(f"language_code: {df_train['language_code'].nunique(dropna=False)}")
print(f"book_format: {df_train['book_format'].nunique(dropna=False)}")
print(f"book_genre: {df_train['book_genre'].nunique(dropna=False)}")

language_code: 76
book_format: 131
book_genre: 11


Como a quantidade de valores distintos em cada coluna é muito grande, e será necessário fazer um tratamento a respeito disso no futuro, vamos criar um novo valor para cada coluna referente a "outros", no qual vamos colocar os valores nulos.

In [13]:
# Verificando e imprimindo as linhas que possuem valores NaN em qualquer coluna
rows_with_nan = df[df.isnull().any(axis=1)]

In [14]:
df_train['language_code'] = df_train['language_code'].fillna('other')
df_train['book_format'] = df_train['book_format'].fillna('other')
df_train['book_genre'] = df_train['book_genre'].fillna('other')

df_val['language_code'] = df_val['language_code'].fillna('other')
df_val['book_format'] = df_val['book_format'].fillna('other')
df_val['book_genre'] = df_val['book_genre'].fillna('other')

df_test['language_code'] = df_test['language_code'].fillna('other')
df_test['book_format'] = df_test['book_format'].fillna('other')
df_test['book_genre'] = df_test['book_genre'].fillna('other')

Agora que os valores nulos das linhas mais significativas foram tratados, vamos tratar os demais valores nulos.

In [15]:
# Verificando todas as linhas que ainda possuem valores nulos
df_train[df_train.isnull().any(axis=1)]

,book_text_reviews_count,language_code,is_ebook,book_rating,book_format,author_id,num_pages,publication_year,book_id,book_ratings_count,book_title,book_genre,author_rating,author_reviews_count,author_name,author_ratings_count
56507,33,ara,False,4.18,Paperback,6421865,190,2013,18721152,178,أوناس,other,4.0,70,NaN,388


Pode-se perceber que na maioria das instâncias nas quais existem valores nulos, esse valores também se encontram na variável alvo `book_rating`. Visto que sem essa variável não é possível treinar nem avaliar o modelo, vamos remover todas as instâncias com valores ausentes nessa coluna.

Além disso, nota-se que grande parte dessas instâncias não possuem dados em nenhuma coluna, logo não são capazes de trazer nenhum valor para o dataset.

In [16]:
df_train = df_train.dropna(subset=['book_rating'])
df_val = df_val.dropna(subset=['book_rating'])
df_test = df_test.dropna(subset=['book_rating'])

In [17]:
# Verificando quais colunas possuem valores nulos
nan_columns = df_train.isnull().sum()
nan_columns = nan_columns[nan_columns > 0]

print(nan_columns)

author_name    1
dtype: int64


Ainda há uma instância com valor ausente na coluna de `author_name`, entretanto como o nome do autor é uma variável que será descartada futuramente para a criação do modelo, pode-se ignorar esse valor nausente.

### Salvando os conjuntos de Treino, Validação e Teste

In [ ]:
df_train.to_csv('../data/clean_data/train.csv', index=False, sep=';', encoding='utf-8', header=True)
df_val.to_csv('../data/clean_data/val.csv', index=False, sep=';', encoding='utf-8', header=True)
df_test.to_csv('../data/clean_data/test.csv', index=False, sep=';', encoding='utf-8', header=True)